In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel

import torch

max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!

dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",

    max_seq_length = max_seq_length,

    dtype = dtype,

    load_in_4bit = load_in_4bit,

    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(

    model,

    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",

                      "gate_proj", "up_proj", "down_proj",],

    lora_alpha = 16,

    lora_dropout = 0, # Supports any, but = 0 is optimized

    bias = "none",    # Supports any, but = "none" is optimized

    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context

    random_state = 3407,

    use_rslora = False,  # We support rank stabilized LoRA

    loftq_config = None, # And LoftQ

)

Unsloth 2024.11.9 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


<a name="Data"></a>

### Data Prep


In [4]:
from datasets import load_dataset

In [5]:
# The data must be formatted with appropriate prompt template first.

# See details here: https://github.com/huggingface/trl/blob/main/examples/scripts/orpo.py



alpaca_prompt = """Instruction: {}



### Context:

{}



### Input:

{}



### Response:

{}"""



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN



def format_prompt(sample):

    instruction = sample["Instruction"]

    context = sample["Context"]

    input       = sample["Text"]

    accepted    = str(sample["Choose"])

    rejected    = str(sample["Reject"])



    # ORPOTrainer expects prompt/chosen/rejected keys

    # See: https://huggingface.co/docs/trl/main/en/orpo_trainer

    sample["prompt"]   = alpaca_prompt.format(instruction, context, input, "")

    sample["chosen"]   = accepted + EOS_TOKEN

    sample["rejected"] = rejected + EOS_TOKEN

    return sample

pass

dataset = load_dataset("MTruc/vn_qa_finance_orpo")["train"]


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/4.66M [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/844k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2922 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/516 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['Instruction', 'Context', 'Text', 'Choose', 'Reject'],
    num_rows: 2922
})

In [7]:
dataset = dataset.map(format_prompt,)

Map:   0%|          | 0/2922 [00:00<?, ? examples/s]

Let's print out some examples to see how the dataset should look like

In [8]:
import pprint

row = dataset[1]

print('INSTRUCTION: ' + '=' * 50)

pprint.pprint(row["prompt"])

print('ACCEPTED: ' + '=' * 50)

pprint.pprint(row["chosen"])

print('REJECTED: ' + '=' * 50)

pprint.pprint(row["rejected"])

INSTRUCTION: ==================================================
('Instruction: Bạn là một chuyên gia tài chính, bạn được cung cấp những chỉ số '
 'tài chính của một mô hình dự đoán giá và những tài liệu tài chính kèm theo. '
 'Người dùng có thể hỏi về những chỉ số này hoặc những câu hỏi khác, nếu người '
 'dùng hỏi về chỉ số hoặc thông tin liên quan đến mô hình, hãy dùng số liệu '
 'được cung cấp để trả lời, còn nếu người dùng hỏi những câu hỏi khác không '
 'liên quan đến mô hình thì không cần dùng những chỉ số tài chính của mô '
 'hình.\n'
 '\n'
 '\n'
 '\n'
 '### Context:\n'
 '\n'
 'Chỉ số của mô hình:\n'
 ' Return (Ann.) [%]: 11.64,\n'
 ' Volatility (Ann.) [%]: 24.26,\n'
 ' Max. Drawdown [%]: -17.11,\n'
 ' Sharpe Ratio: 0.48,\n'
 ' Sortino Ratio: 1.81,\n'
 ' Calmar Ratio: 0.68,\n'
 ' Win Rate [%]: 59.44,\n'
 ' AvgTrade [%]: 0.58,\n'
 ' Generalization Score: 0.66,\n'
 ' Tỉ lệ dự đoán gần đây [%]: 10.\n'
 '\n'
 '\n'
 '\n'
 '### Input:\n'
 '\n'
 'Giao dịch theo phương pháp khớp lệnh ở 

In [9]:
# Enable reward modelling stats

from unsloth import PatchDPOTrainer

PatchDPOTrainer()

<a name="Train"></a>

### Train the model

In [10]:
from trl import ORPOConfig, ORPOTrainer

from unsloth import is_bfloat16_supported



orpo_trainer = ORPOTrainer(

    model = model,

    train_dataset = dataset,

    processing_class = tokenizer,

    args = ORPOConfig(

        max_length = max_seq_length,

        max_prompt_length = max_seq_length//2,

        max_completion_length = max_seq_length//2,

        per_device_train_batch_size = 2,

        gradient_accumulation_steps = 4,

        beta = 0.1,

        logging_steps = 1,

        optim = "adamw_8bit",

        lr_scheduler_type = "linear",

        #max_steps = 10, # Change to num_train_epochs = 1 for full training runs

        num_train_epochs = 20,

        fp16 = not is_bfloat16_supported(),

        bf16 = is_bfloat16_supported(),

        output_dir = "outputs",

        report_to = "none", # Use this for WandB etc

    ),

)

Map:   0%|          | 0/2922 [00:00<?, ? examples/s]

Map:   0%|          | 0/2922 [00:00<?, ? examples/s]

Map:   0%|          | 0/2922 [00:00<?, ? examples/s]

In [11]:
orpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,922 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 7,300
 "-____-"     Number of trainable parameters = 11,272,192
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,2.838600,-0.346662,-0.278025,0.500000,-0.068637,-2.780246,-3.466616,-0.033191,-0.001065
2,2.873400,-0.338572,-0.303731,0.500000,-0.034841,-3.037308,-3.385722,-0.093441,-0.063266
3,3.036800,-0.429684,-0.354160,0.250000,-0.075525,-3.541597,-4.296844,0.111445,0.050351
4,3.032300,-0.427639,-0.287993,0.250000,-0.139646,-2.879933,-4.276389,0.275080,0.174517
5,2.913200,-0.434774,-0.369175,0.500000,-0.065599,-3.691750,-4.347744,-0.047732,-0.027142
6,2.875000,-0.320848,-0.293022,0.500000,-0.027826,-2.930215,-3.208475,-0.091278,-0.139205
7,2.848500,-0.357910,-0.290121,0.375000,-0.067789,-2.901208,-3.579101,0.013430,0.032071
8,2.725000,-0.236199,-0.243560,0.625000,0.007361,-2.435600,-2.361991,0.009077,0.072858
9,2.986300,-0.468651,-0.397224,0.125000,-0.071427,-3.972239,-4.686507,0.037824,-0.056517
10,2.811200,-0.304875,-0.258293,0.375000,-0.046582,-2.582929,-3.048753,0.025333,0.064356


TrainOutput(global_step=7300, training_loss=0.9164129473166923, metrics={'train_runtime': 28466.1537, 'train_samples_per_second': 2.053, 'train_steps_per_second': 0.256, 'total_flos': 0.0, 'train_loss': 0.9164129473166923, 'epoch': 19.98631074606434})

<a name="Inference"></a>

### Inference

In [12]:
import random

test_data = load_dataset("MTruc/vn_qa_finance_orpo")["test"]

In [13]:
i = random.randrange(1, 100)

test_data[i]

{'Instruction': 'Bạn là một chuyên gia tài chính, bạn được cung cấp những chỉ số tài chính của một mô hình dự đoán giá và những tài liệu tài chính kèm theo. Người dùng có thể hỏi về những chỉ số này hoặc những câu hỏi khác, nếu người dùng hỏi về chỉ số hoặc thông tin liên quan đến mô hình, hãy dùng số liệu được cung cấp để trả lời, còn nếu người dùng hỏi những câu hỏi khác không liên quan đến mô hình thì không cần dùng những chỉ số tài chính của mô hình.',
 'Context': 'Chỉ số của mô hình:\n Return (Ann.) [%]: 12.9,\n Volatility (Ann.) [%]: 7.68,\n Max. Drawdown [%]: -34.86,\n Sharpe Ratio: 1.68,\n Sortino Ratio: 1.36,\n Calmar Ratio: 0.37,\n Win Rate [%]: 45.89,\n AvgTrade [%]: 1.63,\n Generalization Score: -0.1,\n Tỉ lệ dự đoán gần đây [%]: 80.\n Các thông tin tài chính bổ sung: Calmar Ratio đo lường tỷ lệ giữa lợi nhuận hàng năm và mức sụt giảm lớn nhất (maximum drawdown) mà chiến lược phải trải qua. Công thức tính là \nCalmar Ratio = Return (Ann.) / Maximum Drawdown. Chỉ số này càng

In [14]:
# alpaca_prompt = Copied from above

FastLanguageModel.for_inference(model) # Enable native 2x faster inference



inputs = tokenizer(

[

    alpaca_prompt.format(

        test_data[i]['Instruction'], # instruction

        test_data[i]['Context'], # context,

        test_data[i]['Text'], # input,

        "", # output - leave this blank for generation!

    )

], return_tensors = "pt").to("cuda")



from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)

_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

<|begin_of_text|>Instruction: Bạn là một chuyên gia tài chính, bạn được cung cấp những chỉ số tài chính của một mô hình dự đoán giá và những tài liệu tài chính kèm theo. Người dùng có thể hỏi về những chỉ số này hoặc những câu hỏi khác, nếu người dùng hỏi về chỉ số hoặc thông tin liên quan đến mô hình, hãy dùng số liệu được cung cấp để trả lời, còn nếu người dùng hỏi những câu hỏi khác không liên quan đến mô hình thì không cần dùng những chỉ số tài chính của mô hình.



### Context:

Chỉ số của mô hình:
 Return (Ann.) [%]: 12.9,
 Volatility (Ann.) [%]: 7.68,
 Max. Drawdown [%]: -34.86,
 Sharpe Ratio: 1.68,
 Sortino Ratio: 1.36,
 Calmar Ratio: 0.37,
 Win Rate [%]: 45.89,
 AvgTrade [%]: 1.63,
 Generalization Score: -0.1,
 Tỉ lệ dự đoán gần đây [%]: 80.
 Các thông tin tài chính bổ sung: Calmar Ratio đo lường tỷ lệ giữa lợi nhuận hàng năm và mức sụt giảm lớn nhất (maximum drawdown) mà chiến lược phải trải qua. Công thức tính là 
Calmar Ratio = Return (Ann.) / Maximum Drawdown. Chỉ số này c

<a name="Save"></a>

### Saving, loading finetuned models

To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.



**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [15]:
model.save_pretrained("lora_model") # Local saving

tokenizer.save_pretrained("lora_model")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [16]:
import os

# Đường dẫn đến thư mục checkpoint
checkpoint_dir = "/kaggle/working/outputs"

# Lấy danh sách tất cả các checkpoint
checkpoints = [ckpt for ckpt in os.listdir(checkpoint_dir) if ckpt.startswith("checkpoint-")]

# Tách phần xyz và tìm checkpoint lớn nhất
largest_checkpoint = max(checkpoints, key=lambda ckpt: int(ckpt.split('-')[-1]))

print(f"Largest checkpoint: {checkpoint_dir + largest_checkpoint}")

Largest checkpoint: /kaggle/working/outputscheckpoint-7300


In [17]:
from huggingface_hub import HfApi



username = "MTruc"

MODEL_NAME = "llama3.2-1b-instruct-orpo-v2"

model_path = checkpoint_dir +'/' +largest_checkpoint

api = HfApi(token="")



api.create_repo(

    repo_id = f"{username}/{MODEL_NAME}",

    repo_type="model"

)



api.upload_folder(

    repo_id = f"{username}/{MODEL_NAME}",

    folder_path = model_path

)

  0%|          | 0/6 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/23.2M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MTruc/llama3.2-1b-instruct-orpo-v2/commit/0ae73772385c136501dd74a5aeb5e5c3d89375cb', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0ae73772385c136501dd74a5aeb5e5c3d89375cb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MTruc/llama3.2-1b-instruct-orpo-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='MTruc/llama3.2-1b-instruct-orpo-v2'), pr_revision=None, pr_num=None)